In [1]:
import torch, torchvision
torch.cuda.empty_cache()
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import glob


import os
import ntpath
import numpy as np
import cv2
import random
import itertools
import pandas as pd
from tqdm import tqdm
import urllib
import json
import PIL.Image as Image

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.structures import BoxMode

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 12, 8

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [2]:
data = pd.read_csv('./data/train_solution_bounding_boxes (1).csv')
dataset = []
for info in data.iterrows():
    info = info[1]
    dataset.append({'file_name':info['image'],'x_min':info['xmin'],'y_min':info['ymin'],'x_max':info['xmax'],'y_max':info['ymax']})

In [3]:
data = pd.DataFrame(dataset)

In [4]:
def get_data():
    dataset_dicts = []
    for idx in tqdm(range(len(data))):
        record = {}
        info = data.iloc[idx]
        height,width = cv2.imread(f'./data/training_images/{info["file_name"]}').shape[:2]
        record['file_name']='./data/training_images/' + info['file_name']
        record['height'] = height
        record['width'] = width
        record['image_id'] = idx
        obj = [{
            'bbox':[info['x_min'],info['y_min'],info['x_max'],info['y_max']],
            'bbox_mode':BoxMode.XYXY_ABS,
            'category_id':0,
            'iscrowd':0
        }]
        record['annotations'] = obj
        dataset_dicts.append(record)
    return dataset_dicts

In [5]:
DatasetCatalog.register('data',lambda : get_data())
MetadataCatalog.get('data').set(thing_classes=['car'])
ballon_metadata = MetadataCatalog.get('data')

In [6]:
# torch.randperm(159826, device='cuda')

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("data",)
cfg.DATASETS.TEST = ()   # no metrics implemented for this dataset
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")  # initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 1000
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (ballon)

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

[06/05 20:01:15 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

100%|██████████| 559/559 [00:01<00:00, 354.53it/s]

[06/05 20:01:16 d2.data.build]: Removed 0 images with no usable annotations. 559 images left.
[06/05 20:01:16 d2.data.build]: Distribution of instances among all 1 categories:
|  category  | #instances   |
|:----------:|:-------------|
|    car     | 559          |
|            |              |
[06/05 20:01:16 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in training: [ResizeShortestEdge(short_edge_length=(640, 672, 704, 736, 768, 800), max_size=1333, sample_style='choice'), RandomFlip()]
[06/05 20:01:16 d2.data.build]: Using training sampler TrainingSampler
[06/05 20:01:16 d2.data.common]: Serializing 559 elements to byte tensors and concatenating them all ...
[06/05 20:01:16 d2.data.common]: Serialized dataset takes 0.21 MiB



Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (2, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.


[06/05 20:01:17 d2.engine.train_loop]: Starting training from iteration 0


/home/indika/anaconda3/lib/python3.7/site-packages/detectron2/modeling/roi_heads/fast_rcnn.py:103: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  num_fg = fg_inds.nonzero().numel()


[06/05 20:01:27 d2.utils.events]:  eta: 0:08:42  iter: 19  total_loss: 1.092  loss_cls: 0.6507  loss_box_reg: 0.4157  loss_rpn_cls: 0.01377  loss_rpn_loc: 0.004811  time: 0.5253  data_time: 0.0075  lr: 4.9953e-06  max_mem: 3354M
[06/05 20:01:38 d2.utils.events]:  eta: 0:08:33  iter: 39  total_loss: 1.043  loss_cls: 0.5782  loss_box_reg: 0.4146  loss_rpn_cls: 0.01033  loss_rpn_loc: 0.00578  time: 0.5298  data_time: 0.0027  lr: 9.9902e-06  max_mem: 3354M
[06/05 20:01:49 d2.utils.events]:  eta: 0:08:23  iter: 59  total_loss: 0.9536  loss_cls: 0.4895  loss_box_reg: 0.4593  loss_rpn_cls: 0.01154  loss_rpn_loc: 0.005547  time: 0.5296  data_time: 0.0028  lr: 1.4985e-05  max_mem: 3354M
[06/05 20:01:59 d2.utils.events]:  eta: 0:08:14  iter: 79  total_loss: 0.9137  loss_cls: 0.4115  loss_box_reg: 0.5102  loss_rpn_cls: 0.004258  loss_rpn_loc: 0.005145  time: 0.5329  data_time: 0.0028  lr: 1.998e-05  max_mem: 3354M
[06/05 20:02:11 d2.utils.events]:  eta: 0:08:04  iter: 99  total_loss: 0.8031  loss

In [ ]:
# cfg = get_cfg()
# cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_C4_3x.yaml"))
# cfg.DATASETS.TRAIN = ("data",)
# cfg.DATASETS.TEST = ()   # no metrics implemented for this dataset
# cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_C4_3x.yaml")  # initialize from model zoo
# # cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64 
# cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1

# os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
# trainer = DefaultTrainer(cfg) 
# trainer.resume_or_load(resume=False)
# trainer.train()

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("data", ("bbox",), False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "data")
print(inference_on_dataset(trainer.model, val_loader, evaluator))
# another equivalent way to evaluate the model is to use `trainer.test`

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set the testing threshold for this model
cfg.DATASETS.TEST = ("data", )
predictor = DefaultPredictor(cfg)

In [ ]:
from detectron2.utils.visualizer import ColorMode
import random

index = -1
dataset_dicts = get_data()
for d in random.sample(dataset_dicts, 12):    
    index += 1
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1], metadata=ballon_metadata, scale=0.8)
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    plt.figure(figsize = (14, 10))
    plt.imshow(cv2.cvtColor(v.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))
    plt.savefig(f'./{index}.png')
    plt.show()